# Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# GAN

paper: https://arxiv.org/pdf/1406.2661

<br>

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbFzwr6%2FbtrrLOmiBsp%2F4ZfYfzsuYdAszzsTDvuDdK%2Fimg.png" width=600>

<br>

GAN(Generative Adversarial Network)은 생성적 적대 신경망으로, 두 개의 신경망이 서로 경쟁하는 구조로 이루어짐.

<br>

<font style="font-size:20px"> 구조 </font>

생성자(Generator): 무작위 노이즈를 입력받아 실제와 유사한 가짜 데이터 생성. <br>
판별자(Discriminator): 입력된 데이터가 실제 데이터인지 생성자가 만든 가짜 데이터인지 판별.


<br>

<font style="font-size:20px"> 특징 </font>

두 네트워크는 적대적으로 훈련되며, 생성자는 판별자를 속이기 위해 점점 더 나은 데이터를 생성하려 하고, 판별자는 생성자가 만든 가짜 데이터를 정확하게 구별하려 함. <br>
-> 생성자는 매우 사실적인 데이터를 생성.

## Vanilla GAN

Ian Goodfellow에 의해 처음 제안된 GAN. <br>
generator와 discriminator를 번갈아 가면서 학습하며 아래와 같음 손실함수를 가짐. <br>

<br>

**Discriminator Loss**:
$$
L_D = -\mathbb{E}[\log(D(x))] - \mathbb{E}[\log(1 - D(G(z)))]
$$
where:
- $ D(x) $: 판별자가 실제 이미지를 진짜로 분류하는 확률.
- $ G(z) $: 생성자가 생성한 이미지.

<br>

**Generator Loss**:
  
$$
L_G = -\mathbb{E}[\log(D(G(z)))]
$$
: 생성한 이미지가 판별자에 의해 진짜로 판단되도록 손실을 최소화


#### Mode Collapse

GAN 훈련 과정에서 발생하는 문제로, generator가 제한된 수의 패턴이나 변형만 생성하게 되는 현상. <br>
-> 생성된 이미지나 데이터가 다양성이 부족하고, 특정 유형의 이미지만을 생성

<br>

**모드 붕괴의 원인** <br>
생성기와 판별기 간의 불균형: 생성기가 판별기에 비해 너무 빨리 학습하거나 반대로 판별기가 너무 빨리 학습하는 경우, 생성기가 특정 패턴에만 집중. <br>

손실 함수의 특성: GAN의 손실 함수는 비대칭적일 수 있어, 생성기가 특정 유형의 샘플을 생성하는 데 집중하도록 유도 가능. <br>

네트워크의 용량: 생성기나 판별기가 충분한 용량을 갖추지 못하면, 다양한 모드를 표현하기 어려울 수 있음. <br>

## Practice

In [4]:
class Generator(nn.Module):
    def __init__(self, configs):
        super().__init__()
        self.configs = configs.get('generator')
        self.latent_dim = configs.get('latent_dim', 10)

        self.layer1 = nn.Sequential(
            nn.Linear(self.latent_dim, 64),
            nn.ReLU(),
            nn.Dropout(),
        )

        self.layer2 = nn.Sequential(
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Dropout(),
        )

        self.layer3 = nn.Sequential(
            nn.Linear(256, 28*28),
            nn.ReLU(),
            nn.Dropout(),
        )

        self.sigmoid = nn.Sigmoid()


    def forward(self, z):
        x = self.layer1(z)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.sigmoid(x)

        return x

In [14]:
configs = {
    'generator': {
        'latent_dim' : 10,
    },
    'discriminator': {
        'input_dim' : 28*28
    }
}


In [15]:
generator = Generator(configs)
z = torch.randn(32, 10)

In [24]:
generator

Generator(
  (layer1): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (layer2): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=256, out_features=784, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (sigmoid): Sigmoid()
)

In [17]:
generator(z).reshape(32, 28, 28).shape

torch.Size([32, 28, 28])

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, configs):
        super().__init__()
        self.configs = configs.get('discriminator')
        self.latent_dim = configs.get('input_dim')

        self.layer1 = nn.Sequential(
            nn.Linear(self.latent_dim, 256),
            nn.ReLU(),
            nn.Dropout(),
        )

        self.layer2 = nn.Sequential(
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(),
        )

        self.layer3 = nn.Sequential(
            nn.Linear(64, 1),
            nn.ReLU(),
            nn.Dropout(),
        )

        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.sigmoid(x)

        return x

In [41]:
discriminator = Discriminator(configs)
# z = torch.randn(32, 28*28)
# discriminator(z).shape

TypeError: empty(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got NoneType"

In [22]:
Discriminator

__main__.Discriminator

## Practice